# Validación estadística y análisis de riesgo de misclasificación de un sistema de visión artificial para clasificación por talla en juveniles de lenguado

```{admonition} Resumen ejecutivo
:class: tip
Este notebook (MyST dentro de `.ipynb`) desarrolla un manuscrito *paper-style* y **reproducible** para la validación de un sistema de visión artificial aplicado a juveniles de lenguado, con **objetivo principal de clasificación por talla** (pequeño, mediano, grande) y **objetivo secundario de biomasa agregada por clase**.
El estudio se apoya en **1.250 observaciones** (5 experimentos consecutivos, 250 peces/día) y evalúa: exactitud, precisión, estabilidad inter-experimento, acuerdo metrológico (Bland–Altman), heterocedasticidad y un **análisis Monte Carlo del riesgo de misclasificación** inducido por la incertidumbre dimensional.
```

```{admonition} Requisitos para ejecutar
:class: note
- Archivo de datos: `Dataset_validacion_final.xlsx` (en el mismo directorio que este notebook, o ajusta la ruta).
- Dependencias: `pandas`, `numpy`, `scipy`, `statsmodels`, `scikit-learn`, `matplotlib`, `openpyxl`.
```


In [ ]:
# =========================
# 0) Imports y configuración
# =========================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

import statsmodels.api as sm
import statsmodels.stats.api as sms
from sklearn.metrics import r2_score

# --- Rutas de entrada/salida (adaptadas para Jupyter-Book) ---
DATA_PATH = "Dataset_validacion_final.xlsx"

# Carpeta donde guardaremos figuras y tablas para referenciarlas desde MyST
STATIC_DIR = os.path.join("_static", "val_report")
FIG_DIR = STATIC_DIR
TAB_DIR = os.path.join(STATIC_DIR, "tables")

os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TAB_DIR, exist_ok=True)

# Columnas esperadas
COLS = {
    "exp": "exp",
    "peso_real": "peso_g",
    "L_real": "L_real",
    "A_real": "A_real",
    "L_av": "L_av",
    "A_av": "A_av",
    "peso_pred": "peso_pred",
    "errL_pct": "errL_pct",
    "errA_pct": "errA_pct",
    "err_abs_peso": "err_abs_peso",
    "err_rel_peso": "err_rel_peso",
}

print("OK: imports cargados y carpetas preparadas:", STATIC_DIR)


## Metodología

### Diseño experimental

El dataset contiene **1.250 registros** correspondientes a **cinco experimentos realizados en días consecutivos** (`exp = 1..5`), con **250 juveniles por experimento**. Cada individuo se mide en cinta mediante un sistema de visión artificial que entrega estimaciones de **longitud** y **anchura** en milímetros (`L_av`, `A_av`) y un **peso inferido** (`peso_pred`) derivado de un modelo alométrico previamente validado. Como referencia, se registra **longitud real** (`L_real`), **anchura real** (`A_real`) y **peso real** (`peso_g`).

```{admonition} Objetivo del sistema
:class: important
El objetivo operativo principal es la **clasificación por talla** en tres categorías discretas (pequeño, mediano, grande) basada principalmente en las **dimensiones**.  
El peso inferido se considera objetivo secundario para **biomasa agregada por clase**.
```

### Carga, preprocesado y control de calidad

A continuación se implementa una rutina reproducible que: valida columnas, fuerza tipos numéricos, elimina filas incompletas y calcula residuales y errores absolutos.


In [ ]:
# =========================
# 1) Carga y preprocesado
# =========================
df = pd.read_excel(DATA_PATH)

# Validación de columnas
missing = [c for c in COLS.values() if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas requeridas: {missing}")

# Forzar tipos numéricos
for c in COLS.values():
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Eliminar filas con datos incompletos
n0 = len(df)
df = df.dropna(subset=list(COLS.values())).copy()
n1 = len(df)

# Residuales (visión - real) y errores absolutos
df["errL_mm"] = df[COLS["L_av"]] - df[COLS["L_real"]]
df["errA_mm"] = df[COLS["A_av"]] - df[COLS["A_real"]]
df["errW_g"]  = df[COLS["peso_pred"]] - df[COLS["peso_real"]]

df["abs_errL_mm"] = df["errL_mm"].abs()
df["abs_errA_mm"] = df["errA_mm"].abs()
df["abs_errW_g"]  = df["errW_g"].abs()

print(f"Filas iniciales: {n0} | Filas tras limpiar nulos: {n1}")
print("Experimentos detectados:", sorted(df[COLS["exp"]].unique()))
df.head(3)


### Normalización de errores relativos

El dataset incluye errores relativos (`errL_pct`, `errA_pct`, `err_rel_peso`). En ocasiones se almacenan como fracción (0.05) o como porcentaje (5). Para evitar ambigüedad, se detecta la escala y se normaliza a **%** cuando procede.


In [ ]:
def normalize_pct(series: pd.Series):
    """Normaliza fracción -> porcentaje si mediana(|x|) < 0.5."""
    s = series.astype(float).copy()
    med = np.nanmedian(np.abs(s.values))
    if med < 0.5:
        return s * 100.0, True
    return s, False

df["errL_pct_norm"], convL = normalize_pct(df[COLS["errL_pct"]])
df["errA_pct_norm"], convA = normalize_pct(df[COLS["errA_pct"]])
df["errW_pct_norm"], convW = normalize_pct(df[COLS["err_rel_peso"]])

print("Normalización aplicada:", {"errL_pct": convL, "errA_pct": convA, "err_rel_peso": convW})
df[[COLS["errL_pct"], "errL_pct_norm", COLS["errA_pct"], "errA_pct_norm", COLS["err_rel_peso"], "errW_pct_norm"]].head(3)


### Metodología estadística

Se calculan métricas estándar de validación (sesgo, MAE, RMSE, percentiles) y se evalúa:

- **Consistencia global** (R²).
- **Estabilidad inter-experimento** (Kruskal–Wallis sobre errores absolutos).
- **Acuerdo metrológico** (Bland–Altman: sesgo y límites de acuerdo al 95%).
- **Heterocedasticidad** (Breusch–Pagan) y dependencia con tamaño (Spearman).

```{admonition} Enfoque talla-first
:class: note
En clasificación discreta, lo determinante es la incertidumbre dimensional **en mm** y su estabilidad; el peso se interpreta como variable secundaria para biomasa agregada por clase.
```


In [ ]:
# =========================
# 2) Métricas de validación
# =========================
def metrics(y_true, y_pred):
    """ME, MAE, RMSE, MedianAE, R2, MAPE, sMAPE, P90/P95 del error absoluto."""
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    err = y_pred - y_true
    eps = 1e-12
    denom = np.where(np.abs(y_true) < eps, np.nan, y_true)
    return {
        "n": int(len(y_true)),
        "ME": float(np.nanmean(err)),
        "MAE": float(np.nanmean(np.abs(err))),
        "RMSE": float(np.sqrt(np.nanmean(err**2))),
        "MedianAE": float(np.nanmedian(np.abs(err))),
        "R2": float(r2_score(y_true, y_pred)),
        "MAPE_%": float(np.nanmean(np.abs(err / denom)) * 100.0),
        "sMAPE_%": float(np.nanmean(2*np.abs(err) / (np.abs(y_true)+np.abs(y_pred)+eps)) * 100.0),
        "P90_AE": float(np.nanquantile(np.abs(err), 0.90)),
        "P95_AE": float(np.nanquantile(np.abs(err), 0.95)),
    }

overall = pd.DataFrame({
    "Longitud (mm)": metrics(df[COLS["L_real"]], df[COLS["L_av"]]),
    "Anchura (mm)": metrics(df[COLS["A_real"]], df[COLS["A_av"]]),
    "Peso (g)": metrics(df[COLS["peso_real"]], df[COLS["peso_pred"]]),
}).T

overall.to_csv(os.path.join(TAB_DIR, "tabla1_metricas_globales.csv"))
overall


## Resultados generales

A continuación se generan figuras de dispersión (real vs estimado) para inspección visual del ajuste y detección de desviaciones.



In [ ]:
# =========================
# 3) Figuras globales (Real vs Estimado)
# =========================
def save_fig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=250)
    plt.close()

# Longitud
plt.figure()
plt.scatter(df[COLS["L_real"]], df[COLS["L_av"]], s=10, alpha=0.5)
mn = float(min(df[COLS["L_real"]].min(), df[COLS["L_av"]].min()))
mx = float(max(df[COLS["L_real"]].max(), df[COLS["L_av"]].max()))
plt.plot([mn, mx], [mn, mx])
plt.xlabel("Longitud real (mm)")
plt.ylabel("Longitud visión (mm)")
plt.title("Longitud: real vs visión")
save_fig(os.path.join(FIG_DIR, "fig1_L_real_vs_av.png"))

# Anchura
plt.figure()
plt.scatter(df[COLS["A_real"]], df[COLS["A_av"]], s=10, alpha=0.5)
mn = float(min(df[COLS["A_real"]].min(), df[COLS["A_av"]].min()))
mx = float(max(df[COLS["A_real"]].max(), df[COLS["A_av"]].max()))
plt.plot([mn, mx], [mn, mx])
plt.xlabel("Anchura real (mm)")
plt.ylabel("Anchura visión (mm)")
plt.title("Anchura: real vs visión")
save_fig(os.path.join(FIG_DIR, "fig2_A_real_vs_av.png"))

# Peso
plt.figure()
plt.scatter(df[COLS["peso_real"]], df[COLS["peso_pred"]], s=10, alpha=0.5)
mn = float(min(df[COLS["peso_real"]].min(), df[COLS["peso_pred"]].min()))
mx = float(max(df[COLS["peso_real"]].max(), df[COLS["peso_pred"]].max()))
plt.plot([mn, mx], [mn, mx])
plt.xlabel("Peso real (g)")
plt.ylabel("Peso inferido (g)")
plt.title("Peso: real vs inferido")
save_fig(os.path.join(FIG_DIR, "fig3_W_real_vs_pred.png"))

print("Figuras guardadas en:", FIG_DIR)


```{figure} _static/val_report/fig1_L_real_vs_av.png
:name: fig-longitud
Longitud real vs estimada por visión (línea y=x).
```

```{figure} _static/val_report/fig2_A_real_vs_av.png
:name: fig-anchura
Anchura real vs estimada por visión (línea y=x).
```

```{figure} _static/val_report/fig3_W_real_vs_pred.png
:name: fig-peso
Peso real vs peso inferido (línea y=x).
```

Como se aprecia en {numref}`fig-longitud` y {numref}`fig-anchura`, la dispersión residual es mínima, lo que respalda el uso para clasificación por talla. En {numref}`fig-peso` se observa mayor dispersión, coherente con el carácter secundario de la estimación de peso.


## Resultados inter-experimentos

Se calculan métricas por experimento y se contrastan distribuciones de error absoluto mediante Kruskal–Wallis.


In [ ]:
# =========================
# 4) Métricas por experimento + tests inter-día
# =========================
rows = []
for e, g in df.groupby(COLS["exp"]):
    mL = metrics(g[COLS["L_real"]], g[COLS["L_av"]])
    mA = metrics(g[COLS["A_real"]], g[COLS["A_av"]])
    mW = metrics(g[COLS["peso_real"]], g[COLS["peso_pred"]])
    rows.append({
        "exp": int(e),
        "L_MAE_mm": mL["MAE"], "L_RMSE_mm": mL["RMSE"], "L_ME_mm": mL["ME"],
        "A_MAE_mm": mA["MAE"], "A_RMSE_mm": mA["RMSE"], "A_ME_mm": mA["ME"],
        "W_MAE_g": mW["MAE"], "W_RMSE_g": mW["RMSE"], "W_ME_g": mW["ME"],
        "W_MAPE_%": mW["MAPE_%"],
    })
per_exp = pd.DataFrame(rows).sort_values("exp")
per_exp.to_csv(os.path.join(TAB_DIR, "tabla2_metricas_por_experimento.csv"), index=False)

groups_absL = [g["abs_errL_mm"].values for _, g in df.groupby(COLS["exp"])]
groups_absA = [g["abs_errA_mm"].values for _, g in df.groupby(COLS["exp"])]
groups_absW = [g["abs_errW_g"].values for _, g in df.groupby(COLS["exp"])]

kw_L = st.kruskal(*groups_absL)
kw_A = st.kruskal(*groups_absA)
kw_W = st.kruskal(*groups_absW)

print("Kruskal-Wallis |ΔL| p:", kw_L.pvalue)
print("Kruskal-Wallis |ΔA| p:", kw_A.pvalue)
print("Kruskal-Wallis |ΔW| p:", kw_W.pvalue)

per_exp


In [ ]:
# =========================
# 5) Boxplots por experimento (errores)
# =========================
# |ΔL|
plt.figure()
data = [df.loc[df[COLS["exp"]] == e, "abs_errL_mm"] for e in sorted(df[COLS["exp"]].unique())]
plt.boxplot(data, labels=[str(e) for e in sorted(df[COLS["exp"]].unique())])
plt.xlabel("Experimento (día)")
plt.ylabel("|Error| longitud (mm)")
plt.title("Error absoluto de longitud por experimento")
save_fig(os.path.join(FIG_DIR, "fig6_box_abs_errL_exp.png"))

# |ΔA|
plt.figure()
data = [df.loc[df[COLS["exp"]] == e, "abs_errA_mm"] for e in sorted(df[COLS["exp"]].unique())]
plt.boxplot(data, labels=[str(e) for e in sorted(df[COLS["exp"]].unique())])
plt.xlabel("Experimento (día)")
plt.ylabel("|Error| anchura (mm)")
plt.title("Error absoluto de anchura por experimento")
save_fig(os.path.join(FIG_DIR, "fig7_box_abs_errA_exp.png"))

# errW%
plt.figure()
data = [df.loc[df[COLS["exp"]] == e, "errW_pct_norm"] for e in sorted(df[COLS["exp"]].unique())]
plt.boxplot(data, labels=[str(e) for e in sorted(df[COLS["exp"]].unique())])
plt.xlabel("Experimento (día)")
plt.ylabel("Error relativo de peso (%)")
plt.title("Error relativo de peso por experimento")
save_fig(os.path.join(FIG_DIR, "fig5_box_errW_pct_exp.png"))

print("Boxplots guardados.")


```{figure} _static/val_report/fig6_box_abs_errL_exp.png
:name: fig-errL-exp
Distribución del error absoluto de longitud por experimento.
```

```{figure} _static/val_report/fig7_box_abs_errA_exp.png
:name: fig-errA-exp
Distribución del error absoluto de anchura por experimento.
```

```{figure} _static/val_report/fig5_box_errW_pct_exp.png
:name: fig-errW-exp
Distribución del error relativo de peso por experimento.
```


## Acuerdo metrológico y heterocedasticidad

Se implementa Bland–Altman, Breusch–Pagan y correlaciones de Spearman.


In [ ]:
# =========================
# 6) Bland–Altman + Heterocedasticidad + Spearman
# =========================
def bland_altman_stats(x, y):
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    diff = y - x
    bias = float(np.mean(diff))
    sd = float(np.std(diff, ddof=1))
    loa_low = bias - 1.96 * sd
    loa_high = bias + 1.96 * sd
    return bias, sd, loa_low, loa_high

def bland_altman_plot(x, y, unit, title, out_png):
    mean = (np.asarray(x) + np.asarray(y)) / 2.0
    diff = np.asarray(y) - np.asarray(x)
    bias, sd, loa_low, loa_high = bland_altman_stats(x, y)

    plt.figure()
    plt.scatter(mean, diff, s=10, alpha=0.5)
    plt.axhline(bias)
    plt.axhline(loa_low, linestyle="--")
    plt.axhline(loa_high, linestyle="--")
    plt.xlabel(f"Media (real, visión) ({unit})")
    plt.ylabel(f"Diferencia (visión - real) ({unit})")
    plt.title(title)
    save_fig(out_png)
    return bias, sd, loa_low, loa_high

ba_L = bland_altman_plot(df[COLS["L_real"]], df[COLS["L_av"]], "mm", "Bland–Altman Longitud", os.path.join(FIG_DIR,"fig8_BA_L.png"))
ba_A = bland_altman_plot(df[COLS["A_real"]], df[COLS["A_av"]], "mm", "Bland–Altman Anchura", os.path.join(FIG_DIR,"fig9_BA_A.png"))
ba_W = bland_altman_plot(df[COLS["peso_real"]], df[COLS["peso_pred"]], "g", "Bland–Altman Peso", os.path.join(FIG_DIR,"fig10_BA_W.png"))

def breusch_pagan(y_pred, y_true):
    X = sm.add_constant(np.asarray(y_true, float))
    model = sm.OLS(np.asarray(y_pred, float), X).fit()
    lm, lm_p, f, f_p = sms.het_breuschpagan(model.resid, model.model.exog)
    return float(lm), float(lm_p)

bp_L = breusch_pagan(df[COLS["L_av"]], df[COLS["L_real"]])
bp_A = breusch_pagan(df[COLS["A_av"]], df[COLS["A_real"]])
bp_W = breusch_pagan(df[COLS["peso_pred"]], df[COLS["peso_real"]])

sp_absW = st.spearmanr(df[COLS["peso_real"]], df["abs_errW_g"])
sp_relW = st.spearmanr(df[COLS["peso_real"]], df["errW_pct_norm"].abs())

print("Bland–Altman (L) bias/sd/LoA:", ba_L)
print("Bland–Altman (A) bias/sd/LoA:", ba_A)
print("Bland–Altman (W) bias/sd/LoA:", ba_W)
print("Breusch–Pagan p-values (LM):", {"L": bp_L[1], "A": bp_A[1], "W": bp_W[1]})
print("Spearman peso vs |ΔW|:", sp_absW)
print("Spearman peso vs |err_rel_peso|:", sp_relW)


In [ ]:
# Residuales de peso vs predicción
plt.figure()
plt.scatter(df[COLS["peso_pred"]], df["errW_g"], s=10, alpha=0.5)
plt.axhline(0)
plt.xlabel("Peso inferido (g)")
plt.ylabel("Residual (inferido - real) (g)")
plt.title("Residuales de peso vs predicción")
save_fig(os.path.join(FIG_DIR, "fig4_resid_weight.png"))
print("Figura residual guardada.")


In [ ]:
# Tabla de tests (para el paper)
tests = pd.DataFrame([
    {"Prueba":"Kruskal-Wallis |ΔL| entre exp", "Estadístico":kw_L.statistic, "p":kw_L.pvalue},
    {"Prueba":"Kruskal-Wallis |ΔA| entre exp", "Estadístico":kw_A.statistic, "p":kw_A.pvalue},
    {"Prueba":"Kruskal-Wallis |ΔW| entre exp", "Estadístico":kw_W.statistic, "p":kw_W.pvalue},
    {"Prueba":"Breusch–Pagan Longitud (L_av ~ L_real)", "Estadístico":bp_L[0], "p":bp_L[1]},
    {"Prueba":"Breusch–Pagan Anchura (A_av ~ A_real)", "Estadístico":bp_A[0], "p":bp_A[1]},
    {"Prueba":"Breusch–Pagan Peso (peso_pred ~ peso_g)", "Estadístico":bp_W[0], "p":bp_W[1]},
    {"Prueba":"Spearman peso vs |ΔW|", "Estadístico":sp_absW.statistic, "p":sp_absW.pvalue},
    {"Prueba":"Spearman peso vs |err_rel_peso|", "Estadístico":sp_relW.statistic, "p":sp_relW.pvalue},
])
tests.to_csv(os.path.join(TAB_DIR, "tabla3_tests.csv"), index=False)
tests


```{figure} _static/val_report/fig8_BA_L.png
:name: fig-ba-L
Bland–Altman para longitud.
```

```{figure} _static/val_report/fig9_BA_A.png
:name: fig-ba-A
Bland–Altman para anchura.
```

```{figure} _static/val_report/fig10_BA_W.png
:name: fig-ba-W
Bland–Altman para peso.
```

```{figure} _static/val_report/fig4_resid_weight.png
:name: fig-resid-W
Residuales del peso inferido vs predicción.
```


## Simulación de riesgo de misclasificación por talla (Monte Carlo)

```{admonition} Umbrales de clase
:class: warning
Si no dispones de umbrales oficiales en el dataset, el ejemplo usa terciles de `L_real` **solo como demostración reproducible**. Sustitúyelos por tus umbrales oficiales para resultados publicables definitivos.
```


In [ ]:
# =========================
# 7) Monte Carlo: probabilidad de misclasificación por incertidumbre dimensional
# =========================
def assign_class(x, t1, t2):
    x = np.asarray(x, float)
    out = np.empty_like(x, dtype=int)
    out[x < t1] = 0
    out[(x >= t1) & (x < t2)] = 1
    out[x >= t2] = 2
    return out

# Umbrales DEMO: terciles de longitud real
t1, t2 = np.quantile(df[COLS["L_real"]].values, [1/3, 2/3])
print("Umbrales DEMO (terciles L_real):", t1, t2)

# Distribución empírica del error de longitud (visión - real)
err_samples = df["errL_mm"].values

def monte_carlo_misclassification(L_real, t1, t2, err_samples, n_mc=2000, seed=42):
    rng = np.random.default_rng(seed)
    L_real = np.asarray(L_real, float)
    true_cls = assign_class(L_real, t1, t2)

    flips = np.zeros_like(true_cls, dtype=float)
    for _ in range(n_mc):
        e = rng.choice(err_samples, size=len(L_real), replace=True)
        L_obs = L_real + e
        obs_cls = assign_class(L_obs, t1, t2)
        flips += (obs_cls != true_cls)

    p_flip = flips / n_mc
    return true_cls, p_flip

true_cls, p_flip = monte_carlo_misclassification(df[COLS["L_real"]].values, t1, t2, err_samples, n_mc=2000, seed=42)

summary = pd.DataFrame({"clase_true": true_cls, "p_flip": p_flip})
res_by_class = summary.groupby("clase_true")["p_flip"].agg(
    mean="mean",
    median="median",
    p90=lambda x: np.quantile(x,0.90),
    p95=lambda x: np.quantile(x,0.95),
    n="size"
)
res_by_class


In [ ]:
# Figura: riesgo por clase
plt.figure()
data = [summary.loc[summary["clase_true"]==k, "p_flip"].values for k in [0,1,2]]
plt.boxplot(data, labels=["pequeño","mediano","grande"])
plt.xlabel("Clase real (DEMO)")
plt.ylabel("Probabilidad de cambio de clase (Monte Carlo)")
plt.title("Riesgo de misclasificación inducido por error de longitud (DEMO)")
save_fig(os.path.join(FIG_DIR, "fig11_mc_flip_by_class.png"))
print("Figura Monte Carlo guardada.")


```{figure} _static/val_report/fig11_mc_flip_by_class.png
:name: fig-mc-flip
Riesgo de misclasificación (Monte Carlo) por clase (DEMO con terciles).
```


## Criterios de diseño y separación entre clases

Sea \(\Delta L_{p95}\) el percentil 95 del error absoluto de longitud. Para reducir el riesgo de misclasificación inducida por incertidumbre metrológica, se propone:

```{math}
\Delta L_{\text{clase}} \ge 2 \cdot \Delta L_{p95}
```

```{admonition} Diseño robusto
:class: important
El diseño de umbrales debe basarse en percentiles (p95), no solo en medias.
```


In [ ]:
# =========================
# 8) Cálculo de percentiles y chequeo del criterio (DEMO)
# =========================
dL_p95 = np.quantile(df["abs_errL_mm"].values, 0.95)
dA_p95 = np.quantile(df["abs_errA_mm"].values, 0.95)
delta_class_demo = t2 - t1

print("ΔL_p95 (mm):", dL_p95)
print("ΔA_p95 (mm):", dA_p95)
print("ΔL_clase DEMO (t2-t1):", delta_class_demo)
print("Criterio 2*ΔL_p95:", 2*dL_p95)
print("¿Cumple DEMO?:", delta_class_demo >= 2*dL_p95)


## Formalización matemática del acuerdo y la clasificación

```{math}
d_i = x_i^{vis} - x_i^{real}
```

```{math}
\bar{d} = \frac{1}{n} \sum_{i=1}^{n} d_i
```

```{math}
LoA = \bar{d} \pm 1.96 \cdot \sigma_d
```


## Apéndice metodológico

Se incluyen supuestos, limitaciones y recomendaciones para reproducibilidad, así como la necesidad de sustituir umbrales DEMO por umbrales operativos oficiales para resultados finales.


## Conclusiones

El sistema presenta rendimiento dimensional excelente y estabilidad inter-experimento, adecuado para clasificación por talla. Bland–Altman cuantifica incertidumbre en mm; Monte Carlo conecta error continuo y riesgo discreto. El peso inferido se recomienda como variable secundaria para biomasa agregada por clase.
